INF8770 Technologies multimédias

Polytechnique Montréal

Importation des librairies

In [ ]:
from PIL import Image
import matplotlib.pyplot as py
import math
import os
import shutil

 Matrices de Co-occurrence (MC)

In [ ]:
def getMCDescriptor(imagePath, dx, dy, color1, color2):
  image = Image.open(imagePath)
  #width = len(image)
  #height = len(image[0])
  #result = 0

  #for column in range(0, width):
  #  for row in range(0, height):
  #    for canal in range(0, 3):
  #      if ()


  return image

print(getMCDescriptor())

Modèles Binaires Locaux (LBP)

Lecture des images

In [ ]:
def read_image():
  return Image.open(reference)

In [ ]:
def visual_quality(referencePath, compressedPath):
    reference = py.imread(referencePath).astype('float').astype('uint8')
    compressed = py.imread(compressedPath).astype('float').astype('uint8')
    result = 0
    l = len(reference)
    h = len(reference[0])

    for i in range(0, l):
        for j in range(0, h):
            for c in range(0, 3):
                result += (reference[i][j][c] - compressed[i][j][c]) ^ 2

    result = result * (1 / (l * h * c))

    return 10 * math.log(((255 ^ 2) / result), 10)

Hypothèse 1

In [ ]:
if os.path.exists('./Result_1'):
    shutil.rmtree('./Result_1')

os.mkdir('./Result_1')

for i in range(1, 31):
    reference =f'../../Ressources/images/sample({i}).jpg'
    os.mkdir(f'./Result_1/sample({i})')
    reference_size =  os.stat(reference).st_size

    for j in range(10, 41,10):
        compressed_JPEG = f'Result_1/sample({i})/sample({i}.{j}).jpeg'
        compressed_JP2 = f'Result_1/sample({i})/sample({i}.{j}).jp2'

        image_file = Image.open(reference)

        image_file.save(compressed_JPEG, 'JPEG', quality=j)
        image_file.save(compressed_JP2, 'JPEG2000', quality=j)

        image_file_size = image_file.tell()
        compressed_size_JPEG =  os.stat(compressed_JPEG).st_size
        compressed_size_JP2 =  os.stat(compressed_JP2).st_size

        taux_compression_JPEG = 1 - (compressed_size_JPEG / reference_size)
        taux_compression_JP2 = 1 - (compressed_size_JP2 / reference_size)

        open(f"Result_1/JPEG.csv","a").write(f'{taux_compression_JPEG},{visual_quality(reference, compressed_JPEG)},'.replace(',', ';').replace('.', ','))
        open(f"Result_1/JP2.csv","a").write(f'{taux_compression_JP2},{visual_quality(reference, compressed_JP2)},'.replace(',', ';').replace('.', ','))

    open(f"Result_1/JPEG.csv","a").write('\n')
    open(f"Result_1/JP2.csv","a").write('\n')

C:\Users\Alexandre\AppData\Local\Temp\ipykernel_2728\3534805486.py:11: RuntimeWarning: overflow encountered in scalar subtract
  result += (reference[i][j][c] - compressed[i][j][c]) ^ 2


Hypothèse 2

In [ ]:
if os.path.exists('./Result_2'):
    shutil.rmtree('./Result_2')

os.mkdir('./Result_2')

for i in range(1, 31):
    reference =f'../../Ressources/images/sample({i}).jpg'
    os.mkdir(f'./Result_2/sample({i})')
    reference_size =  os.stat(reference).st_size

    for j in range(10, 41,10):
        compressed_JPEG = f'Result_2/sample({i})/sample({i}.{j}).jpeg'
        compressed_JP2 = f'Result_2/sample({i})/sample({i}.{j}).jp2'

        image_file = Image.open(reference)
        image_file_quantize = image_file.quantize(colors=j).convert('RGB')

        image_file_quantize.save(compressed_JPEG, 'JPEG', quality=50)
        image_file_quantize.save(compressed_JP2, 'JPEG2000', quality=50)

        image_file_size = image_file_quantize.tell()
        compressed_size_JPEG =  os.stat(compressed_JPEG).st_size
        compressed_size_JP2 =  os.stat(compressed_JP2).st_size

        taux_compression_JPEG = 1 - (compressed_size_JPEG / reference_size)
        taux_compression_JP2 = 1 - (compressed_size_JP2 / reference_size)

        open(f"Result_2/JPEG.csv","a").write(f'{taux_compression_JPEG},{visual_quality(reference, compressed_JPEG)},'.replace(',', ';').replace('.', ','))
        open(f"Result_2/JP2.csv","a").write(f'{taux_compression_JP2},{visual_quality(reference, compressed_JP2)},'.replace(',', ';').replace('.', ','))

    open(f"Result_2/JPEG.csv","a").write('\n')
    open(f"Result_2/JP2.csv","a").write('\n')


C:\Users\Alexandre\AppData\Local\Temp\ipykernel_2728\3534805486.py:11: RuntimeWarning: overflow encountered in scalar subtract
  result += (reference[i][j][c] - compressed[i][j][c]) ^ 2
